In [80]:
import numpy as np
import cv2
import numpy
import math
import glob

In [84]:
# 2.1 Corner Detection
w = 11
h = 8
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = np.zeros((w*h,3), np.float32)
objp[:,:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)

objpoints = [] 
imgpoints = [] 
images = glob.glob(r'.\*.bmp')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # 找到棋盘格角点
    ret, corners = cv2.findChessboardCorners(gray, (w,h),None)
    # 如果找到足够点对，将其存储起来
    if ret == True:  
        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        objpoints.append(objp)
        imgpoints.append(corners)
        # 将角点在图像上显示
        cv2.drawChessboardCorners(img, (w,h), corners, ret)
        img = cv2.resize(img, (960, 960))
        cv2.namedWindow("findCorners",cv2.WINDOW_AUTOSIZE)
        cv2.imshow('findCorners',img)
        cv2.waitKey(0)
cv2.destroyAllWindows()

In [85]:
# 2.2 Find the Intrinsic Matrix 
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print(mtx)

[[2.22339353e+03 0.00000000e+00 1.02888550e+03]
 [0.00000000e+00 2.22295242e+03 1.04050529e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [88]:
# 2.3 Find the Extrinsic Matrix 
def eulerAnglesToRotationMatrix(theta, tran) :
    R_x = np.array([[1, 0, 0],
                    [0, math.cos(theta[0]), -math.sin(theta[0])],
                    [0, math.sin(theta[0]), math.cos(theta[0])]])
    
    R_y = np.array([[math.cos(theta[1]), 0, math.sin(theta[1])],
                    [0, 1, 0],
                    [-math.sin(theta[1]), 0, math.cos(theta[1])]])
    
    R_z = np.array([[math.cos(theta[2]), -math.sin(theta[2]), 0],
                    [math.sin(theta[2]), math.cos(theta[2]), 0],
                    [0, 0, 1]])
    
    R = np.dot(R_z, np.dot( R_y, R_x ))
    extrinsic_matrix = np.concatenate((R,tran), axis=1)
    return extrinsic_matrix
print(eulerAnglesToRotationMatrix(rvecs[2], tvecs[2]))

[[-0.81006559 -0.24317244  0.53353623  2.66420558]
 [-0.47211761 -0.26909386 -0.83946022  4.69878569]
 [ 0.34770491 -0.93190968  0.10317768 12.96473439]]


In [87]:
# 2.4 Find the Distortion Matrix 
print(dist)

[[-0.12851724  0.10091855 -0.00062626  0.00043284 -0.02310165]]
